In [388]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeClassifier

# Синтетический датасет

In [479]:
X, y = make_classification()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Модель

In [480]:
tree = DecisionTreeClassifier().fit(X_train, y_train)
print(f"Cross validation score: {np.mean(cross_val_score(tree, X_test, y_test))}")

Cross validation score: 0.85


# Оптимизация

[Black-Box Optimization Challenge, или как подбирать гиперпараметры для моделей](https://habr.com/ru/companies/hsespb/articles/537020/) (Дата обращения 29.03.2025)

План: 
- [x] Строим гауссовский процесс по уже имеющимся наблюдениям $\{x_i, y_i\}$.
- [ ] Сэмплируем какое-то количество кандидатов $n_{cand}$ (в нашем решении $n_{cand}=min(100*D, 5000)$, где $D$ — количество оптимизируемых параметров).
- [ ] Используя построенный гауссовский процесс, мы вычисляем оценки на средние и дисперсии значений в кандидатах.
- [ ] Вычисляем функцию приобретения в этих $n_{cand}$ кандидатах и выбираем 8 с наибольшими значениями.
- [ ] Возвращаем эти точки как наши предложения проверяющему коду.


# Гауссовский процесс

- [Регрессия гауссовского процесса с самого начала](https://habr.com/ru/companies/skillfactory/articles/562892/) (Дата обращения 29.03.2025)
- [\[DeepBayes\] День 4, лекция 2. Гауссовские процессы и байесовская оптимизация](https://youtu.be/PgJMLpIfIc8?si=Xkxg0Ndqox6NNqUp) (Дата обращения 29.03.2025)


Дано:
$$
\begin{align}
    & \mathbf{X} = \{x_1, ..., x_n\} \\
    & \mathbf{f} = \{f_{1}(x_1), ..., f_{n}(x_n)\} = \mathbf{y} = \{y_1, ..., y_n\} \\
\end{align}
$$

Обучение:
$$
\begin{align}
    & p(\mathbf{f}|\mathbf{X}) = \mathcal{N}(\mu, \mathbf{K}) \\
    & f(\mathbf{x}) \sim \mathcal{GP}(\mu(\mathbf{x}), k(\mathbf{x}, \mathbf{x}^{\prime})) \\
    & \mu(\mathbf{x}) = \mathbb{E}(f(\mathbf{x})) \\
    & k(\mathbf{x}, \mathbf{x}^{\prime}) = \mathbb{E}(f(\mathbf{x} - m(\mathbf{x}))f(\mathbf{x}^{\prime} - m(\mathbf{x}^{\prime}))) = cov(\mathbf{x}, \mathbf{x}^{\prime})=\sigma_{f}^{2}\exp\{-\sum_{i=1}^{d}\frac{(x_{i}-x_{i}^{\prime})^2}{2r_{i}^2}\}\text{ - rbf kernel}
\end{align}
$$

Предсказание:
$$
\begin{align}
    & y_* = f_{*}(\mathbf{x}_{*}) + \epsilon_{*} \\
    & p(\mathbf{y}, f_{*}) = \mathcal{N}\left(0, \left[ \begin{matrix} 
        \mathbf{K} + \sigma^2 I_m & \mathbf{k}_* \\ 
        \mathbf{k}_*^{\top} & \mathbf{K}_{**} 
    \end{matrix} \right]\right) \\
    & \mu_\ast = \mathbf{k}_\ast^\mathrm{T} [\mathbf{K} + \sigma^2 \mathbf{I}_m]^{-1} \mathbf{y}, \\
    & \sigma_\ast^2 = \bar{K}_{\ast\ast} - \mathbf{k}_\ast^\mathrm{T} [\mathbf{K} + \sigma^2 \mathbf{I}_m]^{-1} \mathbf{k}_\ast

\end{align}
$$

In [ ]:
class GaussianProcess:
    def __init__(self, sigma=1, r=1, noise=0.1):
        self.sigma = sigma # Предполагаемое распределение
        self.r = r # Данный параметр масштабирует значения ковариационной функции
        self.noise = noise # Ввожу шум для борьбы с вырожденными ковариационными матрицами 

    def rbf_kernel(self, xi, xj, sigma=1.0, r=1.0):
        return sigma**2 * np.exp( -np.sum( (xi - xj)**2 / (2 * r**2) ) ) # Абсолютно гладкое гауссовское ядро
    
    def cov(self, X1, X2=None):
        if X2 is None:
            X2 = X1
        return np.array(
            [self.rbf_kernel(x1, x2, self.sigma, self.r) for x1 in X1 for x2 in X2]
        ).reshape( (len(X1), len(X2)) )
    
    def fit(self, X_train, y_train):
        self.X_train = np.array(X_train)
        self.y_train = np.array(y_train) 
        return self

    def predict(self, X_test):
        X_test = np.array(X_test)
        K = self.cov(self.X_train) # Считаем ковариационные матрицы трейна с самим собой
        K_ss = self.cov(X_test) # Считаем ковариационные матрицы теста с самим собой
        K_s = self.cov(self.X_train, X_test) # Считаем ковариационные матрицы трейна и тестом
        K_inv = np.linalg.inv(K + self.noise**2 * np.eye(len(self.X_train))) # Обратная матрица с регуляризационным членом
        mu_s = K_s.T @ K_inv @ self.y_train # Предсказываем среднее
        cov_s = K_ss - K_s.T @ K_inv @ K_s # Предсказываем дисперсии
        return (mu_s, np.diag(cov_s))

In [478]:
X_GP, y_GP = make_regression(n_features=1)
X_train_GP, X_test_GP, y_train_GP, y_test_GP = train_test_split(X_GP, y_GP, test_size=0.5)

model_GP = GaussianProcess().fit(X_train_GP, y_train_GP)
mu, var = model_GP.predict(X_test_GP)

r2_score(y_test_GP, mu)

0.9997494920668502